In [2]:
from rsnautils import *

In [3]:
def dcm_tfm(fn): 
    fn = Path(fn)
    try:
        x = fn.dcmread()
        fix_pxrepr(x)
    except Exception as e:
        print(fn,e)
        raise SkipItemException
    if x.Rows != 512 or x.Columns != 512: x.zoom_to((512,512))
    return x.scaled_px

In [5]:
fns = list(df_comb.fname.values)
dest = path/'train_tif'
dest.mkdir(exist_ok=True)
bs=512

In [83]:
# fns2 = set([o.with_suffix('').name for o in dest.ls()])
# fns3 = set([Path(o).with_suffix('').name for o in fns])
# fns4 = [o for o in fns3 if o not in fns2]
# fns5 = [path_trn/Path(o).with_suffix('.dcm') for o in fns4]

In [6]:
dsrc = DataSource(fns, [[dcm_tfm],[os.path.basename]])
dl = TfmdDL(dsrc, bs=bs, num_workers=2)

In [7]:
ext = '.tif'
def dest_fname(fname): return dest/Path(fname).with_suffix(ext)

def save_cropped_jpg(o, dest):
    fname,px = o
    px.save_tif16(dest_fname(fname), bins=bins)

In [86]:
tfm = AffineCoordTfm(size=256)

In [8]:
def process_batch(pxs, fnames, n_workers=4):
#     pxs = tfm(TensorImage(pxs.unsqueeze(1)).cuda()).cpu().squeeze(1)
#     pxs = pxs.cuda()
#     masks = pxs.mask_from_blur(dicom_windows.brain)
#     bbs = mask2bbox(masks)
#     gs = crop_resize(pxs, bbs, 352).cpu().squeeze()
    parallel(save_cropped_jpg, zip(fnames, pxs), n_workers=n_workers, progress=False, dest=dest)

In [13]:
%time process_batch(*dl.one_batch(), n_workers=8)

CPU times: user 2.02 s, sys: 1.86 s, total: 3.87 s
Wall time: 6.39 s


In [14]:
for b in progress_bar(dl): process_batch(*b, n_workers=6)

In [ ]:
fns = dest.ls()

In [ ]:
for i,(ax,fn) in enumerate(zip(subplots(2,4)[1].flat,fns)):
    jpgfn = dest_fname(fn).name
    show_image(Image.open(jpgfn), ax=ax)

In [ ]:
Image.open(jpgfn).shape